In [1]:
import gymnasium as gym
import numpy as np
import math
import os
import configparser
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

import matplotlib.pyplot as plt




In [2]:
from src.hpc_env import HPCenv
from src.validation import Validation

In [3]:
WORKLOAD_PATH = "data/workloads/lublin_256.swf"

# Load config with explicit path and typed parsing
config = configparser.ConfigParser()
config_path = os.path.join(os.getcwd(), 'config_file', 'config.ini')
config.read(config_path)

GAE_LAMBDA = config.getfloat('training', 'gae_lambda')
GAMMA = config.getfloat('training', 'gamma')
EPISODE_LENGTH = config.getint('training', 'episode_length')

## Model training

In [4]:
""" env = HPCenv(workload_path=WORKLOAD_PATH, config=config)

env = ActionMasker(env, mask_fn) 
env = Monitor(env)

model = MaskablePPO("MlpPolicy", env, verbose=1,
                    gamma=GAMMA,
                    gae_lambda=GAE_LAMBDA,
                    seed=42,
                    n_epochs=100
                    )
model.learn(total_timesteps=100000)  """

' env = HPCenv(workload_path=WORKLOAD_PATH, config=config)\n\nenv = ActionMasker(env, mask_fn) \nenv = Monitor(env)\n\nmodel = MaskablePPO("MlpPolicy", env, verbose=1,\n                    gamma=GAMMA,\n                    gae_lambda=GAE_LAMBDA,\n                    seed=42,\n                    n_epochs=100\n                    )\nmodel.learn(total_timesteps=100000)  '

## Model validation